In [ ]:
from managers.history_manager import HistoryManager
from binance.client import Client
from dotenv import load_dotenv
import os

load_dotenv()

client = Client(api_key=os.getenv("BINANCE_API_KEY"), api_secret=os.getenv("BINANCE_API_SECRET"))


hm = HistoryManager(
    client=client,
    start_str="1 Jan, 2023",
    symbol="BTCUSDT",
    interval="1h",
    timelag=16
)


TypeError: HistoryManager.__init__() missing 1 required positional argument: 'start_str'